In [6]:
import pandas
print(pandas.__file__)

c:\Users\alber\anaconda3\envs\bioenv\lib\site-packages\pandas\__init__.py


In [7]:
import gene_tools
import scipy
import pandas
from scipy.stats import fisher_exact


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
import gene_tools
import os


In [3]:
from gene_tools import foldersLoad, cleanDic, compute_scores, geneScores, NA_filtering,evaluate_trait_scores,evaluate_OR
from Scoring.scomet import Mean, Max, Min, Median, pca

root_path = os.path.join('data', '00_BPStart/00_gene_prioritization')

folders = ['eQTL_GWAS_blood', 'Exome', 'GWAS','pQTL_GWAS']
raw_data = foldersLoad(root_path, folders)
merged = pd.read_csv(r"data\00_BPStart\merged_drug_all_db_with_hgnc_TTD.tsv", sep="\t")
dic_clean = cleanDic(raw_data)


📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\BMD_method_eQTL_GWAS_blood.tsv
📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\PD_method_eQTL_GWAS_blood.tsv
📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\EPILEPSY_method_eQTL_GWAS_blood.tsv
📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\ENDOMETRIOSIS_FIG_UKBB_method_eQTL_GWAS_blood.tsv
📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\SBP_method_eQTL_GWAS_blood.tsv
📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\BIPOLAR_method_eQTL_GWAS_blood.tsv
📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\IBD_consortium_method_eQTL_GWAS_blood.tsv
📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\MDD_method_eQTL_GWAS_blood.tsv
📄 Attempting to load: data\00_BPStart/00_gene_prioritization\eQTL_GWAS_blood\CAD_consortium_method

In [4]:
from Scoring.scomet import Mean, Max, Min, Median, pca, Product

ldl_clean = dic_clean["LDL"]

Max(ldl_clean)
Mean(ldl_clean)
Median(ldl_clean)
Min(ldl_clean)
Product(ldl_clean)

score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"]
drug_targets = set(merged[merged['trait'] == 'LDL'].loc[merged[merged['trait'] == 'LDL']["Sum"] >= 3,"EnsemblId"])   
                                                                                                             
for col in score_cols:
    top = ldl_clean.loc[ldl_clean[col] < 0.5]
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")
    
for col in score_cols:
    top = ldl_clean.sort_values(by=col, ascending=True).head(int(0.01 * len(ldl_clean)))
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")


    
len(ldl_clean["GWAS_percentile"].dropna())

ldl_clean

Prioscore_mean: 20.00% of top 1% are drug targets
Prioscore_max: 7.69% of top 1% are drug targets
Prioscore_min: 6.01% of top 1% are drug targets
Prioscore_median: 22.22% of top 1% are drug targets
Prioscore_product: 9.35% of top 1% are drug targets
Prioscore_mean: 4.41% of top 1% are drug targets
Prioscore_max: 2.94% of top 1% are drug targets
Prioscore_min: 6.37% of top 1% are drug targets
Prioscore_median: 4.90% of top 1% are drug targets
Prioscore_product: 5.39% of top 1% are drug targets


,EnsemblId,Trait,eQTL_percentile,Exome_percentile,GWAS_percentile,pQTL_percentile,Prioscore_max,Prioscore_mean,Prioscore_median,Prioscore_min,Prioscore_product
0,ENSG00000000003,LDL,NaN,9.556205,NaN,NaN,9.556205,9.556205,9.556205,9.556205,2.257191
1,ENSG00000000005,LDL,NaN,4.491098,NaN,NaN,4.491098,4.491098,4.491098,4.491098,1.502097
2,ENSG00000000419,LDL,77.370638,15.354770,88.463550,NaN,88.463550,60.396319,77.370638,15.354770,3.854208
3,ENSG00000000457,LDL,66.838884,44.108424,47.128961,NaN,66.838884,52.692090,47.128961,44.108424,3.947274
4,ENSG00000000460,LDL,38.804547,2.965719,42.689049,NaN,42.689049,28.153105,38.804547,2.965719,2.833200
...,...,...,...,...,...,...,...,...,...,...,...
20428,ENSG00000283619,LDL,NaN,10.130215,NaN,NaN,10.130215,10.130215,10.130215,10.130215,2.315523
20429,ENSG00000283623,LDL,NaN,9.348924,NaN,NaN,9.348924,9.348924,9.348924,9.348924,2.235261
20430,ENSG00000283652,LDL,NaN,81.392506,NaN,NaN,81.392506,81.392506,81.392506,81.392506,4.399283
20431,ENSG00000283683,LDL,NaN,90.023917,NaN,NaN,90.023917,90.023917,90.023917,90.023917,4.500075


In [5]:
evaluate_trait_scores(
    df = ldl_clean,
    drug_reference= merged,
    score_columns=["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median", "Prioscore_product"],
    overlap_method="topvalues",
    cutoff=0.01,
    printer=False,
    targetthreshold=3)

{'LDL': {'Prioscore_mean': 4.412,
  'Prioscore_max': 2.941,
  'Prioscore_min': 6.373,
  'Prioscore_median': 4.902,
  'Prioscore_product': 5.392}}

In [6]:


from gene_tools.analysis import evaluate_OR, run_full_trait_pipeline

evaluate_OR(merged,ldl_clean,sum_threshold=3,cutoff = 0.005,score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
            method="topvalues")


[LDL] Prioscore_mean: OR = 1.62, p = 2.8244e-01, drug targets in top 0% = 6
[LDL] Prioscore_max: OR = 1.62, p = 2.8244e-01, drug targets in top 0% = 6
[LDL] Prioscore_min: OR = 3.83, p = 1.0388e-04, drug targets in top 0% = 13
[LDL] Prioscore_median: OR = 3.16, p = 1.4382e-03, drug targets in top 0% = 11
[LDL] Prioscore_product: OR = 3.49, p = 4.0380e-04, drug targets in top 0% = 12


{'LDL': {'Prioscore_mean': [np.float64(1.623), np.float64(0.282), 6],
  'Prioscore_max': [np.float64(1.623), np.float64(0.282), 6],
  'Prioscore_min': [np.float64(3.831), np.float64(0.0), 13],
  'Prioscore_median': [np.float64(3.161), np.float64(0.001), 11],
  'Prioscore_product': [np.float64(3.492), np.float64(0.0), 12]}}

In [7]:
finaldic = run_full_trait_pipeline(dic_clean,drug_reference=merged,scoring_functions = [Mean, Max, Min, Median, Product],
                                   evaluate_or_fn=evaluate_OR,evaluate_score_fn=evaluate_trait_scores,
                        score_kwargs={"cutoff": 0.005,"overlap_method": "topvalues"},or_kwargs = {"score_cols" : ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
                        "method" : "topvalues","cutoff":0.005}
                        ,verbose = False)

finaldic["TC"]


{'data':              EnsemblId Trait  eQTL_percentile  Exome_percentile  \
 0      ENSG00000000003    TC              NaN         13.329790   
 1      ENSG00000000005    TC              NaN          2.684029   
 2      ENSG00000000419    TC        77.776011         35.997874   
 3      ENSG00000000457    TC        45.242826         41.174595   
 4      ENSG00000000460    TC        45.695891          7.036939   
 ...                ...   ...              ...               ...   
 20428  ENSG00000283619    TC              NaN          9.577465   
 20429  ENSG00000283623    TC              NaN         21.413766   
 20430  ENSG00000283652    TC              NaN         73.935690   
 20431  ENSG00000283683    TC              NaN         96.332713   
 20432  ENSG00000283697    TC              NaN         49.460537   
 
        GWAS_percentile  pQTL_percentile  Prioscore_mean  Prioscore_max  \
 0                  NaN              NaN       13.329790      13.329790   
 1                  NaN 

In [ ]:
def get_best_trait_method(finaldic):
    best_trait = None
    best_method = None
    best_or = -float('inf')

    for trait, res in finaldic.items():
        for method, (or_val, p_val, n) in res['OR_eval'].items():
            if or_val > best_or:
                best_or = or_val
                best_trait = trait
                best_method = method

    return best_trait, best_method, best_or

print(f"For 0.5% : ")
finaldic = run_full_trait_pipeline(dic_clean,drug_reference=merged,scoring_functions = [Mean, Max, Min, Median, Product],
                                   evaluate_or_fn=evaluate_OR,evaluate_score_fn=evaluate_trait_scores,
                        score_kwargs={"cutoff": 0.005,"overlap_method": "topvalues"},or_kwargs = {"score_cols" : ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
                        "method" : "topvalues","cutoff":0.005}
                        ,verbose = False)
print(get_best_trait_method(finaldic))



print(f"for 1%: ")
finaldic = run_full_trait_pipeline(dic_clean,drug_reference=merged,scoring_functions = [Mean, Max, Min, Median, Product],
                                   evaluate_or_fn=evaluate_OR,evaluate_score_fn=evaluate_trait_scores,
                        score_kwargs={"cutoff": 0.01,"overlap_method": "topvalues"},or_kwargs = {"score_cols" : ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
                        "method" : "topvalues","cutoff":0.01}
                        ,verbose = False)
print(get_best_trait_method(finaldic))

print(f"for 5%: ")

finaldic = run_full_trait_pipeline(dic_clean,drug_reference=merged,scoring_functions = [Mean, Max, Min, Median, Product],
                                   evaluate_or_fn=evaluate_OR,evaluate_score_fn=evaluate_trait_scores,
                        score_kwargs={"cutoff": 0.05,"overlap_method": "topvalues"},or_kwargs = {"score_cols" : ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
                        "method" : "topvalues","cutoff":0.05}
                        ,verbose = False)

print(get_best_trait_method(finaldic))


For 0.5% : 
('BMD', 'Prioscore_median', inf)
for 1%: 
('EPILEPSY', 'Prioscore_mean', inf)
for 5%: 
('TC', 'Prioscore_min', 7.913)


In [ ]:
from gene_tools import compare_percentiles_boxplot